In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas
from dash import dcc
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine

In [3]:
# engine = create_engine('postgresql://pi:pi@192.168.167.94:5432/Pergola')
engine = create_engine('postgresql://pi:pi@192.168.43.37:5432/Pergola')
df = pandas.read_sql(con=engine, sql='SELECT * FROM public."D69"')
#df.pop("id")
#df.to_csv("JuntekSQL.csv", index=False)

In [5]:
df.to_csv("D69_26_6_2023.csv", index=False)

In [6]:
df

,id,time,power,energy,voltage,current
0,1,2023-04-20 08:13:59.085126+00:00,613.6,0.676,347.30,1.76
1,2,2023-04-20 08:13:59.093941+00:00,613.6,0.676,347.40,1.76
2,3,2023-04-20 08:13:59.147066+00:00,614.8,0.676,347.50,1.76
3,4,2023-04-20 08:13:59.151632+00:00,609.7,0.676,347.90,1.75
4,5,2023-04-20 08:13:59.189213+00:00,613.0,0.677,348.70,1.75
...,...,...,...,...,...,...
5594924,5595219,2023-06-26 18:50:57.431473+00:00,3.3,490.307,54.61,0.06
5594925,5595220,2023-06-26 18:51:12.799540+00:00,3.1,490.307,87.81,0.03
5594926,5595221,2023-06-26 18:51:13.302996+00:00,3.0,490.307,83.90,0.03
5594927,5595222,2023-06-26 18:51:13.808988+00:00,3.1,490.307,73.95,0.04


In [50]:
#df.index = df['time']
#df.reset_index(inplace=True)
#df2.to_csv("D69_26_6_2023_freq10min.csv", index=False)
#df2.reset_index(drop=True)
#df2 = df.groupby(df.time.dt.floor('5Min')).first()
#df2.reset_index(drop=True, inplace=True)
#df2["id"] = df2.index
df2.to_csv("D69_26_6_2023_freq5min.csv", index=False)

In [10]:
df2

,id,time,power,energy,voltage,current
0,1.0,2023-04-20 08:13:59.085126+00:00,613.6,0.676,347.3,1.76
1,715.0,2023-04-20 08:20:08.065138+00:00,722.7,0.732,317.0,2.27
2,1932.0,2023-04-20 08:30:00.478967+00:00,559.3,0.845,347.1,1.61
3,3095.0,2023-04-20 16:28:33.845540+00:00,189.5,1.616,333.1,0.56
4,3273.0,2023-04-20 16:30:00.473157+00:00,179.5,1.620,335.2,0.53
...,...,...,...,...,...,...
5450,5590899.0,2023-06-26 18:10:00.047748+00:00,37.6,490.300,216.2,0.17
5451,5592076.0,2023-06-26 18:20:00.273862+00:00,18.9,490.304,217.0,0.08
5452,5593255.0,2023-06-26 18:30:00.358881+00:00,8.9,490.307,183.4,0.04
5453,5594435.0,2023-06-26 18:40:00.080329+00:00,0.0,490.307,144.7,0.00


In [ ]:
df = pandas.read_csv("JuntekSQL.csv")
df["voltage"] = df["voltage"]/100
df["current"] = df["current"]/100
df["power"] = df["voltage"]*df["current"]
df["charged"] = df["charged"]/100
df["charging"] = df["power"].clip(lower=0)
df["discharging"] = df["power"].clip(upper=0)
df


In [ ]:
green = "rgb(18, 94, 3)"
green_light = "rgb(52, 130, 38)"
red = 'rgba(255, 0, 0, 0.5)'
orange = 'rgb(237, 159, 43)'
blue = "rgb(4, 72, 207)"

fig = go.Figure()

mask = df["power"] >= 0
mask2 = df["power"] <= 0
fig.add_trace(go.Scatter(x=df["time"], y=df["charging"], name="charging", fill='tozeroy', fillcolor=green, line=dict(color=green)))
fig.add_trace(go.Scatter(x=df["time"], y=df["discharging"], name="discharging", fill='tozeroy', fillcolor=red, line=dict(color=red)))
fig.add_trace(go.Scatter(x=df["time"], y=df["charged"], name="charged", yaxis="y2", line=dict(color=orange)))
fig.add_trace(go.Scatter(x=df["time"], y=df["voltage"], name="voltage", yaxis="y3", line=dict(color=blue)))
#fig.add_trace(go.Scatter(x=df["time"], y=[400000, 500000, 600000], name="yaxis4 data", yaxis="y4"))

# Create axis objects
fig.update_layout(xaxis=dict(domain=[0.0, 0.95], showgrid=False), 
                  yaxis=dict(titlefont=dict(color=green), tickfont=dict(color=green), range=[-200, 1000], showgrid=False),
                  yaxis2=dict(titlefont=dict(color=orange), tickfont=dict(color=orange), range=[-26, 130], tickvals = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                              anchor="free", overlaying="y", side="right", position=0.97, showgrid=False),
                  yaxis3=dict(titlefont=dict(color=blue), tickfont=dict(color=blue), range=[0, 29.5], tickvals = [22, 23, 24, 25, 26, 27, 28, 29],
                              anchor="x",  overlaying="y", side="right", showgrid=False),
                 )
                  #yaxis4=dict(title="yaxis4 title", titlefont=dict(color="#9467bd"), tickfont=dict(color="#9467bd"), anchor="free", overlaying="y",side="right", position=0.85))
fig.update_layout(template="plotly_dark")
# Update layout properties
fig.update_layout(title_text="",height=1000)

fig.show()